# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [5]:
#Your code here
import pandas as pd
import numpy as np
import seaborn as sns


df = pd.read_csv('homepage_actions.csv', index_col=0)
df

,id,group,action
timestamp,,,
2016-09-24 17:42:27.839496,804196,experiment,view
2016-09-24 19:19:03.542569,434745,experiment,view
2016-09-24 19:36:00.944135,507599,experiment,view
2016-09-24 19:59:02.646620,671993,control,view
2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...
2017-01-18 09:11:41.984113,192060,experiment,view
2017-01-18 09:42:12.844575,755912,experiment,view
2017-01-18 10:01:09.026482,458115,experiment,view


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8188 entries, 2016-09-24 17:42:27.839496 to 2017-01-18 10:24:08.629327
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8188 non-null   int64 
 1   group   8188 non-null   object
 2   action  8188 non-null   object
dtypes: int64(1), object(2)
memory usage: 255.9+ KB


In [11]:
df['id'].value_counts()

id
750053    2
389447    2
283040    2
918794    2
208714    2
         ..
467818    1
629924    1
820336    1
424916    1
461199    1
Name: count, Length: 6328, dtype: int64

In [8]:
df['group'].value_counts()

group
control       4264
experiment    3924
Name: count, dtype: int64

In [8]:
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


In [9]:
df['action'].value_counts()

action
view     6328
click    1860
Name: count, dtype: int64

In [9]:
cid = set(df[df['action'] == 'click']['id'].unique())
vid = set(df[df['action'] == 'view']['id'].unique())
len_cid = len(cid)
len_vid = len(vid)

print("number of clicks without views: {}".format(len_cid))
print("number of clicks with views: {}".format(len_vid))

number of clicks without views: 1860
number of clicks with views: 6328


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [10]:
#Your code here
df['control'] = 1
df.head()

,id,group,action,control
timestamp,,,,
2016-09-24 17:42:27.839496,804196,experiment,view,1
2016-09-24 19:19:03.542569,434745,experiment,view,1
2016-09-24 19:36:00.944135,507599,experiment,view,1
2016-09-24 19:59:02.646620,671993,control,view,1
2016-09-24 20:26:14.466886,536734,experiment,view,1


In [29]:
control = df[df['group'] =='control'].pivot(index='id', columns='action')
control = control.fillna(value=0)

experiment = df[df['group']=='experiment'].pivot(index='id', columns='action')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996


TypeError: can only concatenate str (not "int") to str

In [23]:
control

group          control     
action    click     view   click view
id                                   
182994  control  control     1.0  1.0
183089        0  control     0.0  1.0
183248  control  control     1.0  1.0
183515        0  control     0.0  1.0
183524        0  control     0.0  1.0
...         ...      ...     ...  ...
936786        0  control     0.0  1.0
937003        0  control     0.0  1.0
937073        0  control     0.0  1.0
937108        0  control     0.0  1.0
937217  control  control     1.0  1.0

[3332 rows x 4 columns]

In [22]:
control.columns

MultiIndex([(  'group', 'click'),
            (  'group',  'view'),
            ('control', 'click'),
            ('control',  'view')],
           names=[None, 'action'])

In [14]:
import numpy as np
import scipy.stats as stats

def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [18]:
p_value_welch_ttest(control['click'], experiment['click'])

KeyError: 'click'

In [36]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3332 entries, 182994 to 937217
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   (group, click)    3332 non-null   object 
 1   (group, view)     3332 non-null   object 
 2   (control, click)  3332 non-null   float64
 3   (control, view)   3332 non-null   float64
dtypes: float64(2), object(2)
memory usage: 130.2+ KB


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [27]:
#Your code here
control_rate = control.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

TypeError: can only concatenate str (not "int") to str

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [26]:
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)


actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

NameError: name 'control_rate' is not defined

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [25]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score) 
print(p_val)

NameError: name 'z_score' is not defined

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.